## Use BeanTech Dataset via API

In [ ]:
import numpy as np
from PIL import Image
from torchvision.transforms import ToPILImage

from anomalib.data.btech import BTech, BTechDataset
from anomalib.data.utils import InputNormalizationMethod, get_transforms

### Torch Dataset

In [ ]:
BTechDataset??

To create `BTechDataset` we need to create the albumentations object that applies transforms to the input image.

In [ ]:
get_transforms??

In [ ]:
image_size = (256, 256)
transform = get_transforms(image_size=256, normalization=InputNormalizationMethod.NONE)

#### Classification Task

In [ ]:
# BTechDataset Classification Train Set
btech_dataset_classification_train = BTechDataset(
    root="../../datasets/BTech",
    category="01",
    transform=transform,
    split="train",
    task="classification",
)
btech_dataset_classification_train.setup()
btech_dataset_classification_train.samples.head()

In [ ]:
sample = btech_dataset_classification_train[0]
sample.keys(), sample["image"].shape

As can be seen above, when we choose `classification` task and `train` split, the dataset only returns `image`. This is mainly because training only requires normal images and no labels. Now let's try `test` split for the `classification` task

In [ ]:
# BTech Classification Test Set
btech_dataset_classification_test = BTechDataset(
    root="../../datasets/BTech",
    category="01",
    transform=transform,
    split="test",
    task="classification",
)
btech_dataset_classification_test.setup()
sample = btech_dataset_classification_test[0]
sample.keys(), sample["image"].shape, sample["image_path"], sample["label"]

where a classification test sample returns `image`, `image_path` and `label`. `image_path` is used to extract the filename when saving images.

#### Segmentation Task

It is also possible to configure the BTech dataset for the segmentation task, where the dataset object returns image and ground-truth mask.

In [ ]:
# BTech Segmentation Train Set
btech_dataset_segmentation_train = BTechDataset(
    root="../../datasets/BTech",
    category="01",
    transform=transform,
    split="train",
    task="segmentation",
)
btech_dataset_segmentation_train.setup()
btech_dataset_segmentation_train.samples.head()

The above dataframe stores all the necessary information regarding the dataset. `__getitem__` method returns the corresponding information depending on the task type or train/test split.

In [ ]:
# BTech Segmentation Test Set
btech_dataset_segmentation_test = BTechDataset(
    root="../../datasets/BTech",
    category="01",
    transform=transform,
    split="test",
    task="segmentation",
)
btech_dataset_segmentation_test.setup()
sample = btech_dataset_segmentation_test[20]
sample.keys(), sample["image"].shape, sample["mask"].shape

Let's visualize the image and the mask...

In [ ]:
# img = Image.open(sample["image_path"]).resize(image_size)
img = ToPILImage()(sample["image"].clone())
msk = ToPILImage()(sample["mask"]).convert("RGB")

Image.fromarray(np.hstack((np.array(img), np.array(msk))))

### DataModule

So far, we have shown the Torch Dateset implementation of BTech AD dataset. This is quite useful to get a sample. However, when we train models end-to-end fashion, we do need much more than this such as downloading the dataset, creating train/val/test/inference dataloaders. To handle all these, we have the PyTorch Lightning DataModule implementation, which is shown below

In [ ]:
btech_datamodule = BTech(
    root="../../datasets/BTech",
    category="01",
    image_size=256,
    train_batch_size=32,
    eval_batch_size=32,
    num_workers=8,
    task="segmentation",
    normalization=InputNormalizationMethod.NONE,
)
btech_datamodule.setup()

In [ ]:
# Train images
i, data = next(enumerate(btech_datamodule.train_dataloader()))
data.keys(), data["image"].shape

In [ ]:
# Test images
i, data = next(enumerate(btech_datamodule.test_dataloader()))
data.keys(), data["image"].shape, data["mask"].shape

As can be seen above, creating the dataloaders is pretty straghtforward, which could be directly used for training/testing/inference. We could visualize samples from the dataloaders as well.

In [ ]:
img = ToPILImage()(data["image"][0].clone())
msk = ToPILImage()(data["mask"][0]).convert("RGB")

Image.fromarray(np.hstack((np.array(img), np.array(msk))))